![Los Angeles skyline](la_skyline.jpg)

Los Angeles, California 😎. The City of Angels. Tinseltown. The Entertainment Capital of the World! 

Known for its warm weather, palm trees, sprawling coastline, and Hollywood, along with producing some of the most iconic films and songs. However, as with any highly populated city, it isn't always glamorous and there can be a large volume of crime. That's where you can help!

You have been asked to support the Los Angeles Police Department (LAPD) by analyzing crime data to identify patterns in criminal behavior. They plan to use your insights to allocate resources effectively to tackle various crimes in different areas.

## The Data

They have provided you with a single dataset to use. A summary and preview are provided below.

It is a modified version of the original data, which is publicly available from Los Angeles Open Data.

# crimes.csv

| Column     | Description              |
|------------|--------------------------|
| `'DR_NO'` | Division of Records Number: Official file number made up of a 2-digit year, area ID, and 5 digits. |
| `'Date Rptd'` | Date reported - MM/DD/YYYY. |
| `'DATE OCC'` | Date of occurrence - MM/DD/YYYY. |
| `'TIME OCC'` | In 24-hour military time. |
| `'AREA NAME'` | The 21 Geographic Areas or Patrol Divisions are also given a name designation that references a landmark or the surrounding community that it is responsible for. For example, the 77th Street Division is located at the intersection of South Broadway and 77th Street, serving neighborhoods in South Los Angeles. |
| `'Crm Cd Desc'` | Indicates the crime committed. |
| `'Vict Age'` | Victim's age in years. |
| `'Vict Sex'` | Victim's sex: `F`: Female, `M`: Male, `X`: Unknown. |
| `'Vict Descent'` | Victim's descent:<ul><li>`A` - Other Asian</li><li>`B` - Black</li><li>`C` - Chinese</li><li>`D` - Cambodian</li><li>`F` - Filipino</li><li>`G` - Guamanian</li><li>`H` - Hispanic/Latin/Mexican</li><li>`I` - American Indian/Alaskan Native</li><li>`J` - Japanese</li><li>`K` - Korean</li><li>`L` - Laotian</li><li>`O` - Other</li><li>`P` - Pacific Islander</li><li>`S` - Samoan</li><li>`U` - Hawaiian</li><li>`V` - Vietnamese</li><li>`W` - White</li><li>`X` - Unknown</li><li>`Z` - Asian Indian</li> |
| `'Weapon Desc'` | Description of the weapon used (if applicable). |
| `'Status Desc'` | Crime status. |
| `'LOCATION'` | Street address of the crime. |

In [137]:
# Re-run this cell
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import datetime

crimes = pd.read_csv("crimes.csv", parse_dates=["Date Rptd", "DATE OCC"], dtype={"TIME OCC": str})
crimes.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status Desc,LOCATION
0,220314085,2022-07-22,2020-05-12,1110,Southwest,THEFT OF IDENTITY,27,F,B,NaN,Invest Cont,2500 S SYCAMORE AV
1,222013040,2022-08-06,2020-06-04,1620,Olympic,THEFT OF IDENTITY,60,M,H,NaN,Invest Cont,3300 SAN MARINO ST
2,220614831,2022-08-18,2020-08-17,1200,Hollywood,THEFT OF IDENTITY,28,M,H,NaN,Invest Cont,1900 TRANSIENT
3,231207725,2023-02-27,2020-01-27,0635,77th Street,THEFT OF IDENTITY,37,M,H,NaN,Invest Cont,6200 4TH AV
4,220213256,2022-07-14,2020-07-14,0900,Rampart,THEFT OF IDENTITY,79,M,B,NaN,Invest Cont,1200 W 7TH ST


Which hour has the highest frequency of crimes? Store as an integer variable called peak_crime_hour. 

In [138]:
# Get an hour from Time OCC
# Convert to and integer

crimes['time_hour'] = crimes['TIME OCC'].str.slice(0,2).astype('int32')

# Group by time_hour to be able to aggregate
# Use count as aggregation to count the crime occurence
# Select any column that does not have null values - DR_NO is enough
# Sort the data descending and get the 1st record as the maximum

peak_crime_hour = crimes.groupby(['time_hour']).count()['DR_NO'].reset_index().sort_values(by = 'DR_NO', ascending = False).head(1)['time_hour'].iloc[0]

peak_crime_hour

12

Which area has the largest frequency of night crimes (crimes committed between 10pm and 3:59am)? Save as a string variable called peak_night_crime_location.

In [139]:
# Create a proper time value to perform datetime operations
crimes['time_occ_value'] = pd.to_datetime(
    crimes['TIME OCC'].str.slice(0,2) + ':' + crimes['TIME OCC'].str.slice(2, 4),
    format = '%H:%M').dt.time

In [140]:
# Set the lower and upper time limits
time_start = dt.time(22,0,0)
time_end = dt.time(3,59,0)

# Since the range crosses the midnight, need to use OR statement
# Otherwise, mathematically it is a wrong condition

crimes_night = crimes[ (crimes['time_occ_value'] >= time_start) | (crimes['time_occ_value'] <= time_end ) ]

# Filter for the 1st record from sorted field containing a count of crime cases
peak_night_crime_location = crimes_night.groupby(['AREA NAME']).count()['DR_NO'].reset_index().sort_values(by = 'DR_NO', ascending = False).head(1)['AREA NAME'].iloc[0]

peak_night_crime_location

'Central'

Identify the number of crimes committed against victims of different age groups. Save as a pandas Series called victim_ages, with age group labels "0-17", "18-25", "26-34", "35-44", "45-54", "55-64", and "65+" as the index and the frequency of crimes as the values.

In [141]:
def f_age_group(age):
    if age <= 17:
        return '0-17'
    elif 17 < age <= 25:
        return '18-25'
    elif 25 < age <= 34:
        return '26-34'
    elif 34 < age <= 44:
        return '35-44'
    elif 44 < age <= 54:
        return '45-54'
    elif 54 < age <= 64:
        return '55-64'
    elif age >= 65:
        return '65+'
    else:
        return 'weird age'

crimes['age_group'] = crimes['Vict Age'].apply(f_age_group)

victim_ages = crimes.groupby('age_group').count()['DR_NO'].rename('frequency_of_crimes')
victim_ages

age_group
0-17      4528
18-25    28291
26-34    47470
35-44    42157
45-54    28353
55-64    20169
65+      14747
Name: frequency_of_crimes, dtype: int64